## This notebook is designed to be used in ArcGIS Pro

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
import os
import zipfile
import arcpy

#need to add zip for geocommons file
#need to also set file path variables which need to be strings

## One file path in here needs to be set, otherwise everything is relative

### Change gdb_folder

In [3]:
#First I'm going to set file path locations that are easy to change for different users' machines
#We'll use these later
gdb_folder = r"C:\Users\KOlso\Documents\GIS_5571_Lab_1"
shapefile_zip_path = os.path.join(gdb_folder, "mngeo_shapefile.zip")
gdb_final = os.path.join(gdb_folder, "olso5581_Lab1.gdb")
gdb_name = "olso5581_Lab1.gdb"

#download_folder = r"C:\Users\KOlso\Downloads" #can delete if test works

#I'm also going to make my file geodatabase now so it's ready at the end
arcpy.management.CreateFileGDB(gdb_folder, gdb_name)

<Result 'C:\\Users\\KOlso\\Documents\\GIS_5571_Lab_1\\olso5581_Lab1.gdb'>

In [4]:
#I'm going to put all my request urls here at the top so they're in one place
#in theory if I needed to do this same lab again but with different data,
#I could just change this cell and otherwise keep things the same


#NDAWN request - here I'm grabbing the Minnesota stations in the network, 
#along with the average temp for Sept 1 this year
ndawn_url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=174&station=118&station=87&station=124&station=226&station=219&station=227&station=184&station=2&station=220&station=223&station=183&station=156&station=70&station=173&station=185&station=187&station=119&station=4&station=82&station=225&station=120&station=71&station=103&station=116&station=114&station=115&station=61&station=181&station=60&station=122&station=5&station=182&station=117&station=6&station=222&station=92&station=123&station=95&station=148&variable=wdavt&ttype=weekly&quick_pick=&begin_date=2024-09-01&count=1"

#MN Geocommons request
#This is the link to a counties dataset
mngeo_url = "https://gisdata.mn.gov//api/3/action/package_show?id=d199886f-474f-4fe0-aba4-c7071ab92a35"

#esri ArcGIS Online REST API
#built query to return data in WGS 84 (4326) and in geojson format
esri_url = "https://gis.hennepin.us/arcgis/rest/services/HennepinData/BOUNDARIES/MapServer/4/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"


In [5]:
#NDAWN request - here I'm grabbing the Minnesota stations in the network, 
#along with the average temp for Sept 1 this year

#ndawn_url = "https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=174&station=118&station=87&station=124&station=226&station=219&station=227&station=184&station=2&station=220&station=223&station=183&station=156&station=70&station=173&station=185&station=187&station=119&station=4&station=82&station=225&station=120&station=71&station=103&station=116&station=114&station=115&station=61&station=181&station=60&station=122&station=5&station=182&station=117&station=6&station=222&station=92&station=123&station=95&station=148&variable=wdavt&ttype=weekly&quick_pick=&begin_date=2024-09-01&count=1"

#method without requests, pandas can read csv directly into dataframe
ndawn_df = pd.read_csv(ndawn_url, skiprows=[0,1,2,4])
ndawn_sedf = pd.DataFrame.spatial.from_xy(df=ndawn_df, x_column="Longitude", y_column="Latitude", sr=4326)

#confirm spatial reference and take a peek at data
print(ndawn_sedf.spatial.sr)
ndawn_sedf.head()


{'wkid': 4326}


,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Number Missing,Number Estimated,SHAPE
0,Ada,47.32119,-96.51406,910,2024,9,1,60.161,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
1,Alvarado,48.24594,-97.02153,809,2024,9,1,61.180,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -97...."
2,Becker,45.34399,-93.85014,942,2024,9,1,63.109,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,Campbell,46.06494,-96.37014,987,2024,9,1,62.335,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96...."
4,Clarissa,46.11155,-94.90583,1304,2024,9,1,60.735,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."


In [6]:
#now we'll grab a counties dataset from the MN Geospatial Commons, we'll use this to get an idea of
#the spatial distribution of the NDAWN stations that happen to be in MN

#first call to api to get the directory to our url
mngeo_res = requests.get(mngeo_url)

#turn that data into json format
mngeo_json_out = mngeo_res.json()

#get the url by following the json structure to the right spot - by specifying 2 under resrouces,
# I am pointing us to a shapefile, but there were other options
mngeo_dl_url = mngeo_json_out['result']['resources'][2]['url']

#just to test
print(mngeo_dl_url)

#create another request variable we can use to get actual data
mngeo_res2 = requests.get(mngeo_dl_url)

https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/shp_bdry_counties.zip


In [7]:
#save my zipped folder in my specified path from above

with open(shapefile_zip_path, 'wb') as f:
    f.write(mngeo_res2.content)

In [8]:
#extract the contents of that zipped folder

extracted_folder = os.path.join(gdb_folder, "mngeo_shapefile")
with zipfile.ZipFile(shapefile_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

In [9]:
# Add path to the specific .shp file we want
shapefile_path = os.path.join(extracted_folder, "County_Boundaries_in_Minnesota.shp")

# Load the shapefile into a spatially enabled DataFrame for the spatial join
sedf_counties = pd.DataFrame.spatial.from_featureclass(shapefile_path)

# Check the spatial reference and preview the data
print(sedf_counties.spatial.sr)
sedf_counties.head()

C:\Users\KOlso\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\KOlso\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


{'wkid': 26915, 'latestWkid': 26915}


,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,Red Lake,63,125,659508,2,160725.772889,1121198845.29,"{""rings"": [[[254267.48900000006, 5317428.819],..."
1,1,Hennepin,27,053,659472,M,189829.833881,1570490144.08,"{""rings"": [[[460790.98000000045, 5009684.501],..."
2,2,Stearns,73,145,659517,3,280898.930397,3598882111.63,"{""rings"": [[[333638.16220000014, 5071100.27899..."
3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25,"{""rings"": [[[419160.2740000002, 5045580.789000..."
4,4,Murray,51,101,659496,8,173803.04716,1864170293.52,"{""rings"": [[[302069.1983000003, 4857932.176000..."


In [10]:
#26915 is what we would expect based on the metadata - that's the wkid for UTM 15N

#but now we need to reproject to WGS 84 (wkid 4326)
sedf_counties.spatial.project(spatial_reference = 4326)

#confirm the spatial reference and preview the table again (probably should remove the above preview)
print(sedf_counties.spatial.sr)
sedf_counties.head()

{'wkid': 4326, 'latestWkid': 4326}


,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,Red Lake,63,125,659508,2,160725.772889,1121198845.29,"{""rings"": [[[-96.29203868018837, 47.9629636673..."
1,1,Hennepin,27,053,659472,M,189829.833881,1570490144.08,"{""rings"": [[[-93.49957384649944, 45.2395687982..."
2,2,Stearns,73,145,659517,3,280898.930397,3598882111.63,"{""rings"": [[[-95.1397848063617, 45.77343063417..."
3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25,"{""rings"": [[[-94.03581736621345, 45.5590780377..."
4,4,Murray,51,101,659496,8,173803.04716,1864170293.52,"{""rings"": [[[-95.46245846222979, 43.8479537270..."


In [11]:
#For the Esri REST API call, we'll grab a dataset that is all the cities in Hennepin County

#esri_url called from earlier cell
esri_res = requests.get(esri_url)

#calling the esri response as json and then converting to feature set
henn_cities = FeatureSet.from_geojson(esri_res.json())

#converting feature set to sedf
henn_sedf = henn_cities.sdf

#Check the spatial reference and preview the data
print(henn_sedf.spatial.sr)
henn_sedf.head()

{'wkid': 4326, 'latestWkid': 4326}


,OBJECTID,FEATURECODE,NAME_TXT,NAME,MUNIC_NO,STATUTE_272_162,Shape.STArea(),Shape.STLength(),SHAPE
0,1,213,MEDINA,MEDINA - 80,80,ALL DIV & COMBO,70153713.42627838,37382.782212733706,"{""rings"": [[[-93.63044742089062, 45.0655585184..."
1,2,213,LONG LAKE,LONG LAKE - 72,72,NO 272.162,2476331.327574715,11017.936519123832,"{""rings"": [[[-93.58823297655248, 44.9892878985..."
2,3,213,CHAMPLIN,CHAMPLIN - 50,50,272.162,22671569.579575375,24406.204779400796,"{""rings"": [[[-93.33052254513477, 45.1525100104..."
3,4,213,ST. BONIFACIUS,ST. BONIFACIUS - 95,95,NO 272.162,2679906.840667625,8974.283400754415,"{""rings"": [[[-93.74300004266519, 44.9015424870..."
4,5,213,FT. SNELLING TERR.,FT. SNELLING TERR. - 41,41,NO 272.162,7426424.36522725,20739.84548682531,"{""rings"": [[[-93.18962944688867, 44.8969376503..."


In [12]:
#I want to see how many stations from my ndawn extract are in each county. 

#first I'll join the two spatially enabled data frames
#This now gives me a record of each station, with the added knowledge of what county it is in
joined_sedf = ndawn_sedf.spatial.join(sedf_counties)

#changed df display options to confirm I had the full selection (and I knew it was a small set)
pd.set_option("display.max_rows", None) 
joined_sedf

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp,Number Missing,Number Estimated,SHAPE,index_right,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area
0,Ada,47.321190,-96.514060,910,2024,9,1,60.161,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",82,82,Norman,54,107,659499,2,247097.098624,2272879542.21
1,Waukon,47.325810,-96.132580,1146,2024,9,1,60.634,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",82,82,Norman,54,107,659499,2,247097.098624,2272879542.21
2,Alvarado,48.245940,-97.021530,809,2024,9,1,61.180,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -97....",57,57,Marshall,45,089,659489,2,374169.657687,4698516270.18
3,Stephen,48.456740,-96.853670,834,2024,9,1,62.138,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",57,57,Marshall,45,089,659489,2,374169.657687,4698516270.18
4,Becker,45.343990,-93.850140,942,2024,9,1,63.109,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93....",3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25
5,Campbell,46.064940,-96.370140,987,2024,9,1,62.335,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",56,56,Wilkin,84,167,659528,4,270716.067488,1947396512.59
6,Rothsay,46.471290,-96.322630,1138,2024,9,1,62.347,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",56,56,Wilkin,84,167,659528,4,270716.067488,1947396512.59
7,Wolverton,46.565450,-96.687260,937,2024,9,1,61.516,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",56,56,Wilkin,84,167,659528,4,270716.067488,1947396512.59
8,Clarissa,46.111550,-94.905830,1304,2024,9,1,60.735,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94....",30,30,Todd,77,153,659521,3,212758.477977,2536516254.29
9,Crookston,47.823333,-96.620556,910,2024,9,1,61.231,0,0,"{""spatialReference"": {""wkid"": 4326}, ""x"": -96....",44,44,Polk,60,119,659505,2,458739.971487,5178444386.51


# The table just generated is my merged table for the assignment

In [13]:
#I have a table that inlcudes the weather station location and what county it is in
#but I want to be able to map this eventually, so I want to know how many stations are in each county

# Next I'll count the number of points in each polygon using pandas groupby functionality
#chatgpt helped me figure out how to create and manipulate the dataframe with reset_index 
station_counts = joined_sedf.groupby('COUNTY_NAM').size().reset_index(name='station_count')

# Check the updated polygons DataFrame with point counts
station_counts

,COUNTY_NAM,station_count
0,Becker,1
1,Clay,2
2,Dakota,1
3,Fillmore,1
4,Grant,2
5,Hubbard,1
6,Kittson,2
7,Lake of the Woods,1
8,Marshall,2
9,Morrison,1


In [14]:
#Now that I have the number per county, I can merge the counts back with the original counties sedf
counties_station_counts = sedf_counties.merge(station_counts, 
                                                             left_on='COUNTY_NAM',
                                                             right_on='COUNTY_NAM', 
                                                             how='left')

# added this so if I wanted to map it later, the nans would be 0s instead
counties_station_counts['station_count'].fillna(0, inplace=True)

#We know that the counts are whole numbers, so I want to change the data type to int
#if I don't do this I get an error when concerting to feature class
counties_station_counts['station_count'] = counties_station_counts['station_count'].astype(int)

#make sure the spatial reference is still correct and preview the combined data
print(counties_station_counts.spatial.sr)
counties_station_counts.head()

{'wkid': 4326, 'latestWkid': 4326}


,FID,COUNTY_NAM,COUNTY_COD,COUNTY_FIP,COUNTY_GNI,ATP_CODE,SHAPE_Leng,SHAPE_Area,SHAPE,station_count
0,0,Red Lake,63,125,659508,2,160725.772889,1121198845.29,"{""rings"": [[[-96.29203868018837, 47.9629636673...",0
1,1,Hennepin,27,053,659472,M,189829.833881,1570490144.08,"{""rings"": [[[-93.49957384649944, 45.2395687982...",0
2,2,Stearns,73,145,659517,3,280898.930397,3598882111.63,"{""rings"": [[[-95.1397848063617, 45.77343063417...",0
3,3,Sherburne,71,141,659515,3,163652.509384,1166966708.25,"{""rings"": [[[-94.03581736621345, 45.5590780377...",1
4,4,Murray,51,101,659496,8,173803.04716,1864170293.52,"{""rings"": [[[-95.46245846222979, 43.8479537270...",0


In [15]:
#now to export our new joined sedf to a file geodatabase

#set new variable for file path
joined_path = os.path.join(gdb_final, "counties_station_counts")

#First our combined table
counties_station_counts.spatial.to_featureclass(location = joined_path)


'C:\\Users\\KOlso\\Documents\\GIS_5571_Lab_1\\olso5581_Lab1.gdb\\counties_station_counts'

In [16]:
#then let's add our other dataframes as feature clasess, so we have them all in one place

#set variables for file path
counties_path = os.path.join(gdb_final, "counties")
ndawn_path = os.path.join(gdb_final, "ndawn_stations")
henn_cities_path = os.path.join(gdb_final, "henn_cities")


sedf_counties.spatial.to_featureclass(location = counties_path)
ndawn_sedf.spatial.to_featureclass(location = ndawn_path)
henn_sedf.spatial.to_featureclass(location = henn_cities_path)

'C:\\Users\\KOlso\\Documents\\GIS_5571_Lab_1\\olso5581_Lab1.gdb\\henn_cities'

### Now I can flip over to my map tab to see the data, and continue with analysis and visualization